In [14]:
import os
import pickle
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, f1_score, accuracy_score, roc_curve
from sklearn.metrics import precision_recall_curve
from scipy import stats

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

from hnet import AppendNet

def load_pretrained_model(pre_trained_loc="./PCLR.h5") :
    pre_trained_model = load_model(pre_trained_loc)
    
    return pre_trained_model

def do_bootstrap(pred_vals, trues, threshold=0.5, n=1000):
    auc_list = []
    apr_list = []
    acc_list = []
    f1_list = []
    
    preds = np.array(pred_vals > threshold).astype('float32')
    
    rng = np.random.RandomState(seed=1)
    for _ in range(n):
        idxs = rng.choice(len(trues), size=len(trues), replace=True)
        pred_arr= preds[idxs]
        true_arr = trues[idxs]
        pred_val_arr = pred_vals[idxs]

        auc = roc_auc_score(true_arr, pred_arr)
        apr = average_precision_score(true_arr, pred_arr)
        acc = accuracy_score(true_arr, np.concatenate(pred_arr))
        f1 = f1_score(true_arr, pred_arr)

        auc_list.append(auc)
        apr_list.append(apr)
        acc_list.append(acc)
        f1_list.append(f1)

    return np.array(auc_list), np.array(apr_list), np.array(acc_list), np.array(f1_list)

def confidence_interval(values, alpha=0.95):
    lower = np.percentile(values, (1-alpha)/2 * 100)
    upper = np.percentile(values, (alpha + (1-alpha)/2) * 100)
    return lower, upper

In [15]:
def get_ecg(df):
    ecgs = []
    for idx in df.index:
        row = df.loc[idx]
        qid = row['QuantaID']
        doc = row['Date_of_Cath']
        fname = f'/storage/shared/apollo/same-day/{qid}_{doc}.csv'
        x = pd.read_csv(fname).values[...,1:].astype(np.float32)
        x /= 1000
        x = x[:4096, :].T
        ecgs.append(x)
        
    ecgs = np.array(ecgs)
    return np.transpose(ecgs, (0,2,1))

def get_data(batch_size=64):
    df_tab = pd.read_csv(os.path.join('/storage/shared/apollo/same-day/tabular_data.csv'))
    train_ids = np.load("./stores/train_ids.npy")
    val_ids = np.load("./stores/val_ids.npy")
    test_ids = np.load("./stores/test_ids.npy")

    train_ids = train_ids[len(train_ids) // 2 :]
    val_ids = val_ids[len(val_ids) // 2 :]
    test_ids = test_ids[len(test_ids) // 2 :]

    train_df = df_tab[df_tab["QuantaID"].isin(train_ids)]
    val_df = df_tab[df_tab["QuantaID"].isin(val_ids)]
    test_df = df_tab[df_tab["QuantaID"].isin(test_ids)]
    print(len(train_df), len(val_df), len(test_df))

    male_ids = np.load("./stores/test_female_ids.npy")
    female_ids = np.load("./stores/test_male_ids.npy")

    male_df = df_tab[df_tab["QuantaID"].isin(male_ids)]
    female_df = df_tab[df_tab["QuantaID"].isin(female_ids)]
    print(len(male_df), len(female_df))
        
    X_train = get_ecg(train_df)
    X_val = get_ecg(val_df)
    X_test = get_ecg(test_df)
    male_test = get_ecg(male_df)
    female_test = get_ecg(female_df)

    y_train = (train_df["PCWP_mean"].values >= 18).astype('float32')
    y_val = (val_df["PCWP_mean"].values >= 18).astype('float32')
    y_test = (test_df["PCWP_mean"].values >= 18).astype('float32')
    y_male = (male_df["PCWP_mean"].values >= 18).astype('float32')
    y_female = (female_df["PCWP_mean"].values >= 18).astype('float32')

    return X_train, y_train, X_val, y_val, X_test, y_test, male_test, y_male, female_test, y_female

In [23]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"

pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=True)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

X_train, y_train, X_val, y_val, X_test, y_test, male_test, y_male, female_test, y_female = get_data()

2442 893 923
1114 711


In [24]:
epochs = 50
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/50
77/77 [==============================] - 6s 46ms/step - loss: 1.8160
Epoch 2/50
77/77 [==============================] - 3s 45ms/step - loss: 0.7773
Epoch 3/50
77/77 [==============================] - 3s 45ms/step - loss: 0.6556
Epoch 4/50
77/77 [==============================] - 4s 46ms/step - loss: 0.6329
Epoch 5/50
77/77 [==============================] - 4s 46ms/step - loss: 0.6140
Epoch 6/50
77/77 [==============================] - 4s 46ms/step - loss: 0.6060
Epoch 7/50
77/77 [==============================] - 3s 45ms/step - loss: 0.6188
Epoch 8/50
77/77 [==============================] - 4s 46ms/step - loss: 0.6054
Epoch 9/50
77/77 [==============================] - 4s 46ms/step - loss: 0.5953
Epoch 10/50
77/77 [==============================] - 4s 47ms/step - loss: 0.5771
Epoch 11/50
77/77 [==============================] - 4s 46ms/step - loss: 0.5904
Epoch 12/50
77/77 [==============================] - 4s 48ms/step - loss: 0.5815
Epoch 13/50
77/77 [==================

In [78]:
epochs = 40
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/40
77/77 [==============================] - 9s 56ms/step - loss: 5.6152
Epoch 2/40
77/77 [==============================] - 4s 56ms/step - loss: 5.5585
Epoch 3/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 4/40
77/77 [==============================] - 4s 56ms/step - loss: 5.5585
Epoch 5/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 6/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 7/40
77/77 [==============================] - 4s 55ms/step - loss: 5.5585
Epoch 8/40
77/77 [==============================] - 4s 58ms/step - loss: 5.5585
Epoch 9/40
77/77 [==============================] - 5s 63ms/step - loss: 5.5585
Epoch 10/40
77/77 [==============================] - 4s 55ms/step - loss: 5.5585
Epoch 11/40
77/77 [==============================] - 4s 55ms/step - loss: 5.5585
Epoch 12/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 13/40
77/77 [==================

In [75]:
epochs = 60
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/60
77/77 [==============================] - 8s 52ms/step - loss: 5.7474
Epoch 2/60
77/77 [==============================] - 5s 59ms/step - loss: 5.3227
Epoch 3/60
77/77 [==============================] - 4s 51ms/step - loss: 5.1879
Epoch 4/60
77/77 [==============================] - 4s 54ms/step - loss: 5.2043
Epoch 5/60
77/77 [==============================] - 4s 52ms/step - loss: 5.2899
Epoch 6/60
77/77 [==============================] - 5s 62ms/step - loss: 5.0397
Epoch 7/60
77/77 [==============================] - 4s 58ms/step - loss: 5.0758
Epoch 8/60
77/77 [==============================] - 4s 56ms/step - loss: 5.0316
Epoch 9/60
77/77 [==============================] - 4s 54ms/step - loss: 4.9309
Epoch 10/60
77/77 [==============================] - 4s 56ms/step - loss: 5.0144
Epoch 11/60
77/77 [==============================] - 4s 53ms/step - loss: 4.8389
Epoch 12/60
77/77 [==============================] - 5s 61ms/step - loss: 4.9376
Epoch 13/60
77/77 [==================

In [72]:
epochs = 75
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/75
77/77 [==============================] - 9s 54ms/step - loss: 5.5017
Epoch 2/75
77/77 [==============================] - 4s 53ms/step - loss: 6.7143
Epoch 3/75
77/77 [==============================] - 4s 54ms/step - loss: 5.8228
Epoch 4/75
77/77 [==============================] - 4s 56ms/step - loss: 5.1356
Epoch 5/75
77/77 [==============================] - 5s 59ms/step - loss: 5.2710
Epoch 6/75
77/77 [==============================] - 4s 54ms/step - loss: 5.2711
Epoch 7/75
77/77 [==============================] - 4s 55ms/step - loss: 5.1118
Epoch 8/75
77/77 [==============================] - 4s 56ms/step - loss: 5.1244
Epoch 9/75
77/77 [==============================] - 4s 54ms/step - loss: 5.1332
Epoch 10/75
77/77 [==============================] - 4s 57ms/step - loss: 4.9833
Epoch 11/75
77/77 [==============================] - 4s 54ms/step - loss: 5.0741
Epoch 12/75
77/77 [==============================] - 5s 59ms/step - loss: 5.0246
Epoch 13/75
77/77 [==================

In [69]:
epochs = 100
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/100
77/77 [==============================] - 9s 54ms/step - loss: 5.9893
Epoch 2/100
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 3/100
77/77 [==============================] - 4s 53ms/step - loss: 5.5596
Epoch 4/100
77/77 [==============================] - 4s 58ms/step - loss: 5.5209
Epoch 5/100
77/77 [==============================] - 4s 53ms/step - loss: 7.3218
Epoch 6/100
77/77 [==============================] - 4s 55ms/step - loss: 7.7875
Epoch 7/100
77/77 [==============================] - 4s 56ms/step - loss: 7.4752
Epoch 8/100
77/77 [==============================] - 4s 53ms/step - loss: 7.8457
Epoch 9/100
77/77 [==============================] - 4s 51ms/step - loss: 7.4221
Epoch 10/100
77/77 [==============================] - 4s 54ms/step - loss: 7.7688
Epoch 11/100
77/77 [==============================] - 4s 57ms/step - loss: 7.5437
Epoch 12/100
77/77 [==============================] - 4s 55ms/step - loss: 7.5054
Epoch 13/100
77/77 [=====

In [13]:
y_pred = full_model.predict(X_test)

29/29 [==============================] - 1s 18ms/step


In [31]:
full_model.save('./PCLR_finetuned_50epc.pb', save_format='tf')

INFO:tensorflow:Assets written to: ./PCLR_finetuned_50epc.pb/assets


INFO:tensorflow:Assets written to: ./PCLR_finetuned_50epc.pb/assets


In [ ]:
# Load saved tf model
loaded_model = tf.keras.models.load_model('./PCLR_finetuned.pb')

# Calculate Classification Performance

In [80]:
'''
40 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.5 0.3351798483206934 0.6648201516793067 0.0
(0.5, 0.5) (0.3054983748645721, 0.36619718309859156) (0.6338028169014085, 0.6945016251354279) (0.0, 0.0)


In [68]:
'''
50 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.7440357825806265 0.5232668349921069 0.7353369447453955 0.6608603541947807
(0.7133001461718788, 0.7726369290474622) (0.47762518462381454, 0.5667567643353442) (0.7053087757313109, 0.7616738894907908) (0.6206778782739778, 0.6980044452192103)


In [77]:
'''
60 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.7135456404202721 0.5147116587344185 0.7492231852654389 0.6176315236672565
(0.6807510494291622, 0.7442520302960727) (0.46605857220647273, 0.5586640078494626) (0.7204767063921993, 0.7757313109425785) (0.5704333546209287, 0.6613424879493288)


In [74]:
'''
75 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.6846302660676242 0.49376882366816766 0.7417410617551463 0.5698951463946115
(0.6541804733822344, 0.7145397546030037) (0.446385566822725, 0.5395325749227305) (0.71397616468039, 0.7681473456121344) (0.5187803713988198, 0.6164033145865742)


In [71]:
'''
100 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.6990473923899231 0.5073281407432844 0.7491733477789816 0.5933705207643143
(0.6673062791552481, 0.7294901458721476) (0.45789507885597597, 0.5547084949493752) (0.7215330444203684, 0.7757313109425785) (0.5428037459117965, 0.6393197278911565)


# Subgroup Performance

In [25]:
'''
50 epoch performance
'''
y_pred = full_model.predict(X_test)
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

29/29 [==============================] - 1s 15ms/step
0.7530461878941631 0.5599256205330069 0.7784431202600216 0.6713002964123366
(0.7234911694200205, 0.7823400541972216) (0.5106484549868601, 0.6069627453412697) (0.7518959913326111, 0.8028439869989166) (0.6306876614691932, 0.7114762674900146)


In [26]:
male_pred = full_model.predict(male_test)
auc_male, apr_male, acc_male, f1_male = do_bootstrap(male_pred, y_male)
print(np.mean(auc_male), np.mean(apr_male), np.mean(acc_male), np.mean(f1_male))
print(confidence_interval(auc_male), confidence_interval(apr_male), confidence_interval(acc_male), confidence_interval(f1_male))

35/35 [==============================] - 1s 18ms/step
0.7507258753965594 0.5393680476546086 0.7739066427289049 0.6610411363480164
(0.7209960206231019, 0.7786899808832602) (0.4929084617692631, 0.5817191733975555) (0.7477558348294434, 0.7998204667863554) (0.6192317920714843, 0.6986132944082641)


In [27]:
female_pred = full_model.predict(female_test)
auc_female, apr_female, acc_female, f1_female = do_bootstrap(female_pred, y_female)
print(np.mean(auc_female), np.mean(apr_female), np.mean(acc_female), np.mean(f1_female))
print(confidence_interval(auc_female), confidence_interval(apr_female), confidence_interval(acc_female), confidence_interval(f1_female))

23/23 [==============================] - 0s 14ms/step
0.7189093939892324 0.5802674796823021 0.7468917018284107 0.6454017769165924
(0.6865776801415433, 0.7523731434920649) (0.5262370606713961, 0.6316654388471022) (0.7144866385372715, 0.7777777777777778) (0.5963359956051386, 0.6918526838966204)


In [35]:
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc.npy', auc)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr.npy', apr)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc.npy', acc)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1.npy', f1)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_male.npy', auc_male)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_male.npy', apr_male)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_male.npy', acc_male)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_male.npy', f1_male)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_female.npy', auc_female)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_female.npy', apr_female)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_female.npy', acc_female)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_female.npy', f1_female)

In [29]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(auc_male, auc_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(auc_male, auc_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 1043.7631107089233
  p-value: 5.520066360748927e-229

Independent t-test:
  Statistic: 44.360812155488276
  p-value: 9.057658498841297e-300


In [30]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(apr_male, apr_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(apr_male, apr_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 837.0163653948283
  p-value: 4.833496299067937e-184

Independent t-test:
  Statistic: -36.16394216400008
  p-value: 9.686798304732356e-221


In [31]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(f1_male, f1_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(f1_male, f1_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 204.180642071072
  p-value: 2.5560427990712206e-46

Independent t-test:
  Statistic: 15.296907303573525
  p-value: 4.895397221767411e-50


In [32]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(acc_male, acc_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(acc_male, acc_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 951.5657351536727
  p-value: 6.058867905842296e-209

Independent t-test:
  Statistic: 40.513877337497824
  p-value: 1.7923794186520421e-262


In [22]:
'''
0.7466383919640586 0.5470243498933457 0.7678234019501625 0.6629406219367823
(0.7171828525946882, 0.776655525633572) (0.4979873922374438, 0.5928792111511485) (0.7410617551462622, 0.7941495124593716) (0.6175962416107382, 0.7026278015945095)

0.764761569484173 0.5482275288751095 0.7754524236983841 0.6778789599935521
(0.7375531821039054, 0.7919339045209898) (0.5001759634677061, 0.5922628080008435) (0.7495287253141831, 0.7998204667863554) (0.6380562658083903, 0.7137246930784827)

0.7462905545013693 0.605554705155347 0.7653867791842476 0.6864282382023428
(0.7144760255433712, 0.7782598256501183) (0.5526925644888443, 0.6578182053424811) (0.7355836849507735, 0.7960618846694796) (0.6389986273922029, 0.7307709447415329)
'''


'\n0.7466383919640586 0.5470243498933457 0.7678234019501625 0.6629406219367823\n(0.7171828525946882, 0.776655525633572) (0.4979873922374438, 0.5928792111511485) (0.7410617551462622, 0.7941495124593716) (0.6175962416107382, 0.7026278015945095)\n\n0.764761569484173 0.5482275288751095 0.7754524236983841 0.6778789599935521\n(0.7375531821039054, 0.7919339045209898) (0.5001759634677061, 0.5922628080008435) (0.7495287253141831, 0.7998204667863554) (0.6380562658083903, 0.7137246930784827)\n\n0.7462905545013693 0.605554705155347 0.7653867791842476 0.6864282382023428\n(0.7144760255433712, 0.7782598256501183) (0.5526925644888443, 0.6578182053424811) (0.7355836849507735, 0.7960618846694796) (0.6389986273922029, 0.7307709447415329)\n'